In [28]:
import requests
import re
import json
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import time

def get_urls(counter=0, per_page=20):
    urls = pd.Series(dtype='object')
    url_template = "https://cse.google.com/cse/element/v1?rsz=filtered_cse&num={per_page}&hl=en&source=gcsc&start={counter}&cselibv=8fa85d58e016b414&cx=003256708414512871905%3A8w113ub4vx0&q=opposition+leader+sajith+premadasa+&safe=off&cse_tok=AB-tC_6-D1uGDnb3FeEwN5U9Tu2z%3A1726596157894&sort=&exp=cc&callback=google.search.cse.api16244&rurl=https%3A%2F%2Fwww.hirunews.lk%2Fenglish%2Fsearch.php%3Fq%3Dopposition%2Bleader%2Bsajith%2Bpremadasa%2B"
    
    # Get the first response to estimate total result count
    response = requests.get(url_template.format(counter=counter, per_page=per_page))
    jsonp_response = response.text
    print(jsonp_response)
    json_data = re.search(r'google\.search\.cse\.api\d{4}\((.*)\)', jsonp_response, re.DOTALL).group(1)
    parsed_data = json.loads(json_data)
    total_results = int(parsed_data['cursor']['estimatedResultCount'])

    # Set up the progress bar
    with tqdm(total=total_results) as pbar:
        while counter < total_results:
            time.sleep(0.5)
            url = url_template.format(counter=counter, per_page=per_page)
            response = requests.get(url)
            jsonp_response = response.text
            try:
                json_data = re.search(r'google\.search\.cse\.api\d{4}\((.*)\)', jsonp_response, re.DOTALL).group(1)
            except AttributeError:
                print(jsonp_response)
                counter += per_page
                continue
            parsed_data = json.loads(json_data)

            df = pd.DataFrame(parsed_data['results'])
            urls += pd.Series(df['url'])

            counter += per_page
            pbar.update(len(df))  # Update the progress bar based on how many results were returned

    return urls



def get_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.select_one('.main-tittle').text
    text = soup.select_one('#article-phara2').text
    return {'title': title, 'text': text}

In [29]:
get_urls()

<html><head><meta http-equiv="content-type" content="text/html; charset=utf-8"/><title>Sorry...</title><style> body { font-family: verdana, arial, sans-serif; background-color: #fff; color: #000; }</style></head><body><div><table><tr><td><b><font face=sans-serif size=10><font color=#4285f4>G</font><font color=#ea4335>o</font><font color=#fbbc05>o</font><font color=#4285f4>g</font><font color=#34a853>l</font><font color=#ea4335>e</font></font></b></td><td style="text-align: left; vertical-align: bottom; padding-bottom: 15px; width: 50%"><div style="border-bottom: 1px solid #dfdfdf;">Sorry...</div></td></tr></table></div><div style="margin-left: 4em;"><h1>We're sorry...</h1><p>... but your computer or network may be sending automated queries. To protect our users, we can't process your request right now.</p></div><div style="margin-left: 4em;">See <a href="https://support.google.com/websearch/answer/86640">Google Help</a> for more information.<br/><br/></div><div style="text-align: cente

AttributeError: 'NoneType' object has no attribute 'group'

In [37]:
# Import the required modules
from googlesearch import search
import pandas as pd

# Define the search query
query = "Sajith Premadasa site:hirunews.lk"

# Perform the search and store the results
search_results = search(query, sleep_interval=2, num_results=5000)

# Convert the search results to a pd.Series
search_results_series = pd.Series(search_results)

# Print the pd.Series
print(search_results_series)

HTTPError: 429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://www.google.com/search%3Fq%3DSajith%2BPremadasa%2Bsite%253Ahirunews.lk%26num%3D5002%26hl%3Den%26start%3D0%26safe%3Dactive&hl=en&q=EgQUoyiDGLaLqLcGIjAW5FFH56dKEnQyAgGr0NZNZ1DdPhqJEwbazL7kc1CckfKNWhCq0knx8bO1jDW0PfcyAXJaAUM

In [7]:
df.head()

,clicktrackUrl,content,contentNoFormatting,title,titleNoFormatting,formattedUrl,unescapedUrl,url,visibleUrl,richSnippet,breadcrumbUrl
0,https://www.google.com/url?client=internal-ele...,"Feb 25, 2024 <b>...</b> Samagi Jana Balawegaya...","Feb 25, 2024 ... Samagi Jana Balawegaya (SJB) ...",&quot;I am a proud father of a baby girl&quot;...,"""I am a proud father of a baby girl"" - Opposit...",https://www.hirunews.lk/.../i-am-a-proud-fathe...,https://www.hirunews.lk/english/363233/i-am-a-...,https://www.hirunews.lk/english/363233/i-am-a-...,www.hirunews.lk,{'cseImage': {'src': 'https://cdn.hirunews.lk/...,"{'host': 'www.hirunews.lk', 'crumbs': ['englis..."
1,https://www.google.com/url?client=internal-ele...,"Mar 21, 2024 <b>...</b> <b>Opposition leader S...","Mar 21, 2024 ... Opposition leader Sajith Prem...",<b>Opposition leader Sajith Premadasa</b> meet...,Opposition leader Sajith Premadasa meets with ...,https://www.hirunews.lk/.../<b>opposition</b>-...,https://www.hirunews.lk/english/365350/opposit...,https://www.hirunews.lk/english/365350/opposit...,www.hirunews.lk,{'cseImage': {'src': 'https://cdn.hirunews.lk/...,"{'host': 'www.hirunews.lk', 'crumbs': ['englis..."
2,https://www.google.com/url?client=internal-ele...,"Jun 25, 2024 <b>...</b> <b>Opposition Leader S...","Jun 25, 2024 ... Opposition Leader Sajith Prem...",<b>Opposition Leader Sajith Premadasa</b> crit...,Opposition Leader Sajith Premadasa criticizes ...,https://www.hirunews.lk/.../<b>opposition</b>-...,https://www.hirunews.lk/english/373612/opposit...,https://www.hirunews.lk/english/373612/opposit...,www.hirunews.lk,{'cseImage': {'src': 'https://cdn.hirunews.lk/...,"{'host': 'www.hirunews.lk', 'crumbs': ['englis..."
3,https://www.google.com/url?client=internal-ele...,"Apr 6, 2024 <b>...</b> ... <b>Opposition Leade...","Apr 6, 2024 ... ... Opposition Leader, Sajith ...",Former SLFP General Secretary Dayasiri Jayasek...,Former SLFP General Secretary Dayasiri Jayasek...,https://www.hirunews.lk/.../former-slfp-genera...,https://www.hirunews.lk/english/366642/former-...,https://www.hirunews.lk/english/366642/former-...,www.hirunews.lk,{'cseImage': {'src': 'https://cdn.hirunews.lk/...,"{'host': 'www.hirunews.lk', 'crumbs': ['englis..."
4,https://www.google.com/url?client=internal-ele...,"Dec 6, 2019 <b>...</b> The UNP led by UNP <b>l...","Dec 6, 2019 ... The UNP led by UNP leader Rani...",Speaker accepts <b>Sajith Premadasa</b> as <b>...,Speaker accepts Sajith Premadasa as opposition...,https://www.hirunews.lk/.../speaker-accepts-<b...,https://www.hirunews.lk/english/229652/speaker...,https://www.hirunews.lk/english/229652/speaker...,www.hirunews.lk,{'cseImage': {'src': 'https://cdn.hirunews.lk/...,"{'host': 'www.hirunews.lk', 'crumbs': ['englis..."
